In [1]:
import pandas as pd
import numpy as np
import chardet
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
filepath = '/Users/andrewlipschultz/Dropbox/My Mac (Andrew’s MacBook Pro)/Documents/GitHub/cs224N_final_project/nytcrosswords.csv'
filepath = 'nytcrosswords.csv'
with open(filepath, 'rb') as file:
    result = chardet.detect(file.read())
    encoding = result['encoding']


In [2]:
df = pd.read_csv(filepath, encoding=encoding)
print(df.head())

relevant_cols = ['Clue', 'Word']
for col in relevant_cols:
    text_data = df[col].apply(str).tolist()
    training_text = "\n".join(text_data)


         Date     Word                                 Clue
0  10/31/2021      PAT  Action done while saying "Good dog"
1  10/31/2021  RASCALS                      Mischief-makers
2  10/31/2021      PEN          It might click for a writer
3  10/31/2021      SEP                             Fall mo.
4  10/31/2021      ECO                Kind to Mother Nature


In [3]:
with open('training_data.txt', 'w', encoding='utf-8') as file:
    file.write(training_text)

# Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')

In [4]:
from transformers import pipeline

In [5]:
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Generate text
text_prompt = ""
generated_text = generator(text_prompt, max_length=100, num_return_sequences=1)
print(generated_text[0]['generated_text'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


To read or to access the full report, please click here.


In [6]:
clues = []
answers = []
lengths = []
with open('output.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()
    for line in lines:
        line = line.strip().strip('"')
        parts = line.rsplit(',', 2)
        if len(parts) == 3:
            clue = parts[0].strip().strip('"')
            answer = parts[1].strip().strip('"')
            length = int(parts[2].strip())
            clues.append(clue)
            answers.append(answer)
            lengths.append(length)
            

In [16]:
for i in range(5):
    length = lengths[i]
    clue = clues[i]
    prompt = f"Crossword clue: '{clue}'. Provide the answer in one word that has a length of {length} characters."
    response = generator(prompt, max_length=50, num_return_sequences=1)
    full_response = response[0]['generated_text']
    print(full_response)
    

    

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Crossword clue: 'Action done while saying, ""Good dog'. Provide the answer in one word that has a length of 3 characters. If you want the Answer to be "Good dog", use the first word of the sequence. (Only 5


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Crossword clue: 'Mischief-makers'. Provide the answer in one word that has a length of 7 characters.

The word 'fraud' and the term 'false-spellcasting' have both been used to define a number of


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Crossword clue: 'It might click for a writer'. Provide the answer in one word that has a length of 3 characters.

If you provide a word with length 5, then the next word starts at 5 characters, and after that word


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Crossword clue: 'Fall mo.'. Provide the answer in one word that has a length of 3 characters. Write the final word in all characters and the word of the last letter in the sequence. This is the first of the three types of
Crossword clue: 'Kind to Mother Nature'. Provide the answer in one word that has a length of 3 characters. (If you could, that would be great, and there is a link). (Try to include a keyword for a word to
